<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/ROBERTA_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pickle
import gensim
import numpy as np
import pandas as pd

import nltk
import time
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from wordcloud import WordCloud, STOPWORDS
from gensim.parsing.preprocessing import stem_text
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

from gensim import corpora, models, similarities
from gensim.models import Word2Vec, TfidfModel, LsiModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.fasttext import FastText

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorboard as tb
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Activation,Dense,Flatten,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.python.framework import ops
#tf.gfile = tf.io.gfile
#tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

import random
import pathlib
import matplotlib.pyplot as plt
AUTOTUNE = tf.data.experimental.AUTOTUNE

def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:
        pickle_object = pickle.load(input)
    return  pickle_object

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
#!mkdir imdb

def pre_process(data):
  _sent=[]
  for i in data:
    sent = word_tokenize(i)
    sent = [w.lower() for w in sent]
    sent = [word for word in sent if word not in stop_words]
    sent = [word for word in sent if word.isalpha()]
    sent = [c for c in sent if not c.isdigit()]
    #sent = [stem_text(w) for w in sent]
    #sent = [wordnet_lemmatizer.lemmatize(w) for w in sent]
    #sent = [word for word in sent if word not in stoplist]
    _sent.append(sent)
  return _sent

from keras.datasets import imdb
data = imdb.load_data(path='/content/drive/My Drive/Movielensdata/ml25m/imdb/data',
    num_words=None,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3)
save_object(data, '/content/drive/My Drive/Movielensdata/ml25m/imdb/data')
word_to_idx_dict = imdb.get_word_index(path="/content/drive/My Drive/Movielensdata/ml25m/imdb/imdb_word_index.json")
n = len(word_to_idx_dict)
print("Number distinct words in reviews = %d \n" % n)

words_freq_list = []
for (k,v) in word_to_idx_dict.items():
  words_freq_list.append((k,v))

sorted_list = sorted(words_freq_list, key=lambda x: x[1])

print("Ten most common words: \n")
print(sorted_list[0:10])

print("\nLast five least common words: \n")
print(sorted_list[-5:])

Using TensorFlow backend.


A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 599dadb1135973df5b59232a0e9a887c so we will re-download the data.
17465344/17464789 [==============================] - 2s 0us/step
Number distinct words in reviews = 88584 

Ten most common words: 

[('the', 1), ('and', 2), ('a', 3), ('of', 4), ('to', 5), ('is', 6), ('br', 7), ('in', 8), ('it', 9), ('i', 10)]

Last five least common words: 

[("pipe's", 88580), ('copywrite', 88581), ('artbox', 88582), ("voorhees'", 88583), ("'l'", 88584)]


In [4]:
def data_module(reviews, name, location, output_location, dict_location, max_seq_length=100):
  if reviews==False:
    dataset1 = pd.read_csv(location)
    dataset = dataset1.iloc[0:1000,:]
  else:
    dataset = load_object(location)
  if name == 'word2vec':
    data_corpus, users_total = data_word2vec(reviews,dataset,max_seq_length)
  elif name == 'doc2vec':
    data_corpus, users_total = data_doc2vec(reviews,dataset,max_seq_length)
  elif name == 'lsi':
    data_corpus, users_total = data_lsi(reviews,dataset,max_seq_length,dict_loc=dict_location)
  elif name == 'tfidf':
    data_corpus, users_total = data_lsi(reviews,dataset,max_seq_length,dict_loc=dict_location)
  elif name == 'glove':
    data_corpus, users_total = data_word2vec(reviews,dataset,max_seq_length)
  elif name == 'hashing':
    data_corpus, users_total = data_hashing(reviews,dataset,max_seq_length)
  elif name == 'cooccur':
    data_corpus, users_total = data_hashing(reviews,dataset,max_seq_length)
  #elif name == 'fasttext':
   # data_corpus, users_total = data_word2vec(dataset,max_seq_length)
  #data_corpus.save('/content/drive/My Drive/Movielensdata/ml25m/data_corpus')
  #print(data_corpus)
  save_object(obj=data_corpus, filename=output_location)
  #return data_corpus, users_total

def data_word2vec(review,dataset,max_seq_length):
  if review==False:
    dataset.sort_values(by=['userId','timestamp'],inplace=True)
    user_total = len(dataset['userId'].unique())
    
    #Selecting the most recent movies rated by each user and padding if necessary
    movie_list = []
    for i in range(user_total):
      list1 = []
      list1 = dataset.loc[dataset['userId'] ==(i+1),['movieId']]['movieId'].tolist()
      if len(list1)>max_seq_length:
        list1 = list1[(len(list1)-max_seq_length):]
      elif len(list1)<max_seq_length:
        list1 = list1+[0 for j in range((max_seq_length-len(list1)))]
        #for j in range((max_seq_length-len(list1))):
        # list1.append(0)
      movie_list.append(list1)

    #Selecting the most recent ratings rated by each user and padding if necessary
    rating_list =[]
    for i in range(user_total):
      list2 = []
      list2 = dataset.loc[dataset['userId'] ==(i+1),['rating']]['rating'].tolist()
      if len(list2)>max_seq_length:
        list2 = list2[(len(list2)-max_seq_length):]
      elif len(list2)<max_seq_length:
        list2 = list2+[0 for j in range((max_seq_length-len(list2)))]
        #for j in range((max_seq_length-len(list2))):
        # list2.append(0)
      rating_list.append(list2)

    #Creating user_id level transpose matrices
    movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(user_total)])
    movies_transpose.index.names = ['userId']
    #print(movies_transpose)

    ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(user_total)])
    ratings_transpose.index.names = ['userId']
    #print(ratings_transpose)

    # Select features from original dataset to form a new dataframe 
    df1 = movies_transpose.iloc[:]# For each row, combine all the columns into one column
    df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1)# Store them in a pandas dataframe
    df_clean = pd.DataFrame({'clean': df2})# Create the list of list format of the custom corpus for gensim modeling 
    sent = [row.split(',') for row in df_clean['clean']]

    return sent, user_total
  else:
    X_train = dataset[0][0]
    Y_train = dataset[0][1]
    X_test, X_val, Y_test, Y_val = train_test_split(dataset[1][0], dataset[1][1], test_size=0.4, shuffle=False)
    
    reverse_word_index = dict([(value, key) for (key, value) in word_to_idx_dict.items()])
    train_reviews=[]
    for j in X_train:
      train_reviews.append(' '.join([reverse_word_index.get(i - 3, '') for i in j]))

    test_reviews=[]
    for j in X_test:
      test_reviews.append(' '.join([reverse_word_index.get(i - 3, '') for i in j]))

    val_reviews=[]
    for j in X_val:
      val_reviews.append(' '.join([reverse_word_index.get(i - 3, '') for i in j]))

    train_sent=pre_process(train_reviews)
    save_object(train_sent,'/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/train_sent')
    test_sent=pre_process(test_reviews)
    save_object(test_sent,'/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/test_sent')
    val_sent=pre_process(val_reviews)
    save_object(val_sent,'/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/val_sent')
    save_object(Y_train, '/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_train')
    save_object(Y_test, '/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_test')
    save_object(Y_val, '/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_val')

    return train_sent, len(train_sent)

def data_doc2vec(review,dataset,max_seq_length):
  Sent, user_total = data_word2vec(review,dataset,max_seq_length)
  tagged_data = []
  tags = []
  
  for i in range(user_total):
    tagged_data = tagged_data + [TaggedDocument(words=Sent[i], tags=[str(i)])]

  return tagged_data, user_total

def data_lsi(review,dataset,max_seq_length,dict_loc):
  Sent, user_total = data_word2vec(review,dataset,max_seq_length)
  dictionary = corpora.Dictionary(Sent)
  #print(dictionary.token2id)
  corpus = [dictionary.doc2bow(text) for text in Sent]
  vocab = list(dictionary.values())
  save_object(vocab,'/content/drive/My Drive/Movielensdata/ml25m/imdb/vocab') 
  dictionary.save(dict_loc)
  print(dictionary)
  #corpus = np.array([[(id, freq) for id, freq in cp] for cp in corp])
  #corpus = gensim.matutils.Dense2Corpus(np.array(Sent),documents_columns=False)

  return corpus, user_total

def data_hashing(review,dataset,max_seq_length):
  Sent, user_total = data_word2vec(review,dataset,max_seq_length)
  corpus = [str(str(doc)[1:-1]) for doc in Sent]
  return corpus, user_total

In [5]:
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-cfgr10gf
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-cfgr10gf
     |████████████████████████████████| 3.0MB 2.7MB/s 
     |████████████████████████████████| 1.1MB 27.9MB/s 
     |████████████████████████████████| 890kB 31.1MB/s 
  Created wheel for transformers: filename=transformers-3.0.2-cp36-none-any.whl size=793805 sha256=89ab370100b17ec396116941d4d08f940b2a4b8d5dad32ac79223eabcf08fc46
  Stored in directory: /tmp/pip-ephem-wheel-cache-xd167n8n/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f122250764f6cc9a93f8e3be04e5f715f359d0e234e42d0916b86fc5f49f0367
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses
tokenizers               0.8.1rc2       
tra

In [6]:
data_module(reviews=True, name='word2vec',max_seq_length=100,location='/content/drive/My Drive/Movielensdata/ml25m/imdb/data',output_location='/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/train_sent', dict_location='/content/drive/My Drive/Movielensdata/ml25m/imdb/dict/tfidf_dict')

In [7]:
bert_vocab = load_object('/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/train_sent')

In [8]:
print(len(bert_vocab))

25000


In [9]:
VOC_FNAME = "train.txt"

with open(VOC_FNAME, "w") as fo:
  for doc in bert_vocab:
    for token in doc:
      fo.write(token+"\n")

In [10]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

#paths = [str(x) for x in Path(".").glob("**/*.txt")]
paths = '/content/train.txt'
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=1, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [11]:
!mkdir BERT
tokenizer.save_model("BERT")

['BERT/vocab.json', 'BERT/merges.txt']

In [12]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = ByteLevelBPETokenizer('/content/BERT/vocab.json', '/content/BERT/merges.txt')

In [13]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [14]:
#tokenizer.encode("'5952','2012','0'")
tokenizer.encode("name time place")

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [15]:
tokenizer.encode("name time place").tokens

['<s>', 'name', 'Ġ', 'time', 'Ġ', 'place', '</s>']

In [16]:
# Check that we have a GPU
!nvidia-smi

Tue Jul 21 15:57:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
import torch
torch.cuda.is_available()

True

In [18]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [19]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./BERT", max_len=512)

In [20]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [21]:
model.num_parameters()
# => 84 million parameters

84095008

In [22]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/content/train.txt",
    block_size=128,
)

CPU times: user 1min 2s, sys: 3.56 s, total: 1min 6s
Wall time: 47.8 s


In [23]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [24]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./BERT",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [25]:
trainer.train()

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


TrainOutput(global_step=48220, training_loss=8.425708667335051)

In [26]:
trainer.save_model("./BERT")

In [27]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BERT",
    tokenizer="./BERT"
)

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:805: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [30]:
#106916
#fill_mask("this movie is <mask>")

In [31]:
#fill_mask("why <mask>")
#1265

In [32]:
sent = ["this movie is nice","didn't like it too boring"]
model.encode(sent)

AttributeError: ignored

In [33]:
sentence_embedding = torch.mean(hidden_states[-1], dim=1).squeeze()

NameError: ignored